In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as wg
from scipy.signal import find_peaks
import sys
sys.path.append("../src")
import importlib
import support
import visualizations
import calibrations
import kinematics
from scipy import signal
importlib.reload(support)
from support import readFileDialog, parse_data_file, getFrame, saveFileDialog
importlib.reload(kinematics)
from kinematics import getKeypointsCoord, getKeypointsPixels, showRowingChainAnglesPlot, inverseKinematicsRowing
importlib.reload(visualizations)
from visualizations import showFrame, keypointsDATAtoFrame
importlib.reload(calibrations)
from calibrations import getMmppInterface, defineReference
%matplotlib widget

In [2]:
file_path_op = readFileDialog("Open Data file (OP)", "data")

In [3]:
data, var_names = parse_data_file(file_path_op)
kp_openpose = data["keypoints"]
md_openpose = data["metadata"]
ang_openpose = data["angles"]
fps_openpose = md_openpose["fps"]
angles_names = md_openpose['angles_names']
t_openpose = np.linspace(0, len(ang_openpose)*(1/fps_openpose), len(ang_openpose))

In [4]:
joint_angle = wg.Dropdown(options=angles_names,
                        description='Angle:',
                        value='Shoulder <- Hip -> Knee',
                        disabled=False)

def plot_2d_comparison(joint_angle):
    angle_idx = angles_names.index(joint_angle)
    plt.close('all')
    plt.figure()
    plt.plot(t_openpose, ang_openpose[:, angle_idx],  label='Openpose')
    angle_name = joint_angle.split('->')[0].split('<-')[-1].replace(" ", "")
    plt.title("2D Comparation for the {}".format(angle_name))
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (Degrees)")
    plt.grid(True)
    plt.legend()
    plt.show()

out = wg.interactive_output(plot_2d_comparison, {'joint_angle': joint_angle})

vbox = wg.VBox([joint_angle, out])
display(vbox)

In [57]:
frame_n_slider = wg.IntSlider(value=0,
                            min=0,
                            max=len(kp_openpose)-1,
                            step=1,
                            description='Frame:',
                            disabled=False,
                            continuous_update=False)
def kp_angle_view(frame_n):
    kp_openpose_xy = getKeypointsCoord(kp_openpose[frame_n], md_openpose['frame_height'])
    showRowingChainAnglesPlot(kp_openpose_xy, anatomical='True')
    angles = inverseKinematicsRowing(kp_openpose_xy, anatomical='True')
    
output_view = wg.interactive_output(kp_angle_view, {'frame_n': frame_n_slider})
display(frame_n_slider, output_view)
print(angles)

IntSlider(value=0, continuous_update=False, description='Frame:', max=119)

Output()

[  5.3663226  -36.77457876  80.63608604 -50.91315547 162.31189146]


In [28]:
output_path = saveFileDialog("Save without extension") + '.sto'
output_path

'C:/Users/victo/Documents/Motion_analysis/OpenSim/rowing.sto'

In [55]:
ang_openpose_anatomical = np.copy(ang_openpose)
ang_openpose_anatomical[:, 0] = 40 - ang_openpose_anatomical[:, 0]
ang_openpose_anatomical[:, 1] = ang_openpose_anatomical[:, 1] - 180
ang_openpose_anatomical[:, 2] = 180 - ang_openpose_anatomical[:, 2]

In [58]:
with open(output_path, 'w+') as f:
    columns_list = ["time", "ankle_angle_r", "hip_flexion_r", "hip_adduction_r", "hip_rotation_r",
                "knee_angle_r", "subtalar_angle_r", "lumbar_extension", "lumbar_bending", "lumbar_rotation"]
    columns_line = ''.join([col + '\t' for col in columns_list])
    op_to_os = {"time": t_openpose,
                "ankle_angle_r": ang_openpose_anatomical[:, 0],
                "hip_flexion_r": ang_openpose_anatomical[:, 2],
                "knee_angle_r": ang_openpose_anatomical[:, 1]}
    header = "Coordinates\nversion=1\nnRows={}\nnColumns={}\ninDegrees=yes\n\nendheader".format(len(ang_openpose_anatomical),
                                                                                               len(columns_list))

    f.write(header)
    f.write('\n')
    f.write(columns_line)
    f.write('\n')
    
    for n in range(len(ang_openpose_anatomical)):
        values_list = []
        for col_name in columns_list:
            if col_name in op_to_os.keys():
                values_list.append(str(op_to_os[col_name][n]))
            else:
                values_list.append("0")
        values_line = ''.join([value + '\t' for value in values_list])
        f.write(values_line)
        f.write('\n')